In [23]:
import site
site.addsitedir('Lib/site-packages')
import getpass
import os
import requests
import ffmpeg
import json
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from lumaai import AsyncLumaAI

In [ ]:
#LUMA
client = AsyncLumaAI(
    auth_token=os.environ.get("LUMAAI_API_KEY"),
)
#OPENAI
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model
#LANGCHAIN
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

## SCHEMAS

In [16]:
#######EXAMPLES DELETE AFTER
class add_schema(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply_schema(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")
#############EXAMPLES^^^^^^

## TOOLS

In [19]:
##############E
@tool("addition_tool",args_schema=add_schema)
def add(a: int, b: int) -> int:
    return a + b


@tool("multiplication_tool",args_schema=multiply_schema)
def multiply(a: int, b: int) -> int:
    return a * b


tools = [add, multiply]


@tool
def generate_vid(vid_prompt: str, vid_duration: int) -> str:
    """ generates a video"""
    generation = client.generations.create(
        prompt=vid_prompt,
        model="ray-2",
        resolution="720p",
        duration=vid_duration
    )
    video_url = generation.assets.video
    # download the video
    response = requests.get(video_url, stream=True)
    with open(f'{generation.id}.mp4', 'wb') as file:
        file.write(response.content)
    return f"File downloaded as {generation.id}.mp4"

#########################################CODE ABOVE THIS
## BINDING
llm_with_tools = llm.bind_tools(tools)

## RUNNING MODEL

In [ ]:
query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)